# Работа с реляционными базыми данных

### План
1. Вспоминаем курс БД: Реляционная базы данных и Язык SQL
1. SQLite3
1. ORM. [SQLAlchemy](http://www.sqlalchemy.org)

### Реляционная базы данных

Реляционные базы данных:
  * Связанная информация, хранящаяся в двумерных таблицах
  * Каждая таблица представляет собой набор полей (колонки) и набор объектов (записей)
  * Порядок записей в таблице произволен, столбцов - фиксирован
  * Идентификация записей осуществляется с помощью первичных ключей

Если коротко, то реляционные базы данных - это таблицы с индексами для быстрого поиска, связанные между собой

### Язык SQL

* [Язык структурированных запросов](https://ru.wikipedia.org/wiki/SQL)
* Стандартизирован, но БД пытаются соответствовать стандарту ANSI не позволяя ему ограничивать их слишком сильно
* Типы: Числа (INTEGER и DECIMAL), строки (CHAR, VARCHAR) и т.д.
* Операторы: SELECT FROM, WHERE, GROUP BY, COUNT, HAVING и т.д.
* Джойны: Inner join, outer join и т.д.
* Подзапросы
* Команды модификации: INSERT, DELETE, UPDATE
* Создание таблиц


### SQLite3

<img src='pictures/sqlite.png' />

* Простейший способ использовать БД
* Вся БД хранится в одном файле
* Отличный способ иметь БД в мобильных приложениях
* Доступ из коммандной строки: sqlite3 filename
* API для Python, с++ и т.д.

Посмотрим как работать с sqlite3 из python, на примере простой базы с пользователями

Для начала создадим таблицы:

In [60]:
import sqlite3

conn = sqlite3.connect('example.db')
cur = conn.cursor() #Объект для исполнения SQL запросов над базой

#Просто пишем SQL запрос для создания таблиц
cur.execute('''
    CREATE TABLE user_types (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        name VARCHAR(255)
    )
''')
cur.execute('''
    CREATE TABLE users (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        first_name VARCHAR(255),
        last_name VARCHAR(255),
        active BOOLEAN,
        profile INTEGER,
        FOREIGN KEY(profile) REFERENCES user_types(id)
    )
''')

conn.commit() # Отправить текущую транзакцию. 
# Если не сделать, то изменения не будут видны в других connect'ax

conn.close() # Не вызывает commit автоматически!

Работать с пустыми таблицами - не очень интересно, давайте заполним их

In [61]:
conn = sqlite3.connect('example.db')
cur = conn.cursor()

#таблиаца user_types
cur.execute('''
    INSERT INTO user_types (name) VALUES ("Student")
''')
cur.execute('''
    INSERT INTO user_types (name) VALUES ("Teacher")
''')

#таблица users
cur.execute('''
    INSERT INTO users (first_name, last_name, active, profile) VALUES 
        ("Maxim", "Popov", 1, (SELECT id FROM user_types WHERE name = "Teacher"))
''')
cur.execute('''
    INSERT INTO users (first_name, last_name, active, profile) VALUES 
        ("Igor", "Orlov", 1, (SELECT id FROM user_types WHERE name = "Teacher"))
''')
cur.execute('''
    INSERT INTO users (first_name, last_name, active, profile) VALUES 
        ("Ivan", "Ivanov", 0, (SELECT id FROM user_types WHERE name = "Student"))
''')
cur.execute('''
    INSERT INTO users (first_name, last_name, active, profile) VALUES 
        ("Petr", "Petrov", 0, (SELECT id FROM user_types WHERE name = "Student"))
''')
cur.execute('''
    INSERT INTO users (first_name, last_name, active, profile) VALUES 
        ("Petr", "Sidorov", 0, (SELECT id FROM user_types WHERE name = "Student"))
''')

conn.commit()

conn.close()

Попробуем теперь зачитать всех студентов

In [62]:
conn = sqlite3.connect("example.db")

def select_all_students(conn):
    query = '''SELECT users.first_name, users.last_name 
               FROM users 
               WHERE profile = (SELECT id FROM user_types WHERE name = "Student")'''
    cur = conn.cursor()
    cur.execute(query)

    rows = cur.fetchall() #Получить все строчки результата выполнения query
    #Это не всегда быстро, особенно если много результатов
    # Кроме того большое потребление памяти, так как возвращается list
    
    #rows = cur.fetchmany(10) # Как fetchall только задаем сколько хотим получить

    for row in rows:
        first_name, last_name = row
        print(first_name, last_name)
        
select_all_students(conn)

Ivan Ivanov
Petr Petrov
Petr Sidorov


In [63]:
def select_all_students_optimized(conn):
    query = '''SELECT users.first_name, users.last_name 
               FROM users 
               WHERE profile = (SELECT id FROM user_types WHERE name = "Student")'''
    cur = conn.cursor()
    cur.execute(query)
    
    #Можно получать результаты по одному
    #row = cur.fetchone()
    #while row is not None:
    #    first_name, last_name = row
    #    print(first_name, last_name)
    #    row = cur.fetchone()
    
    #А лучше просто проитерироваться
    for row in cur:
        first_name, last_name = row
        print(first_name, last_name)

In [64]:
select_all_students_optimized(conn)

Ivan Ivanov
Petr Petrov
Petr Sidorov


Попробуем сделать учителя Maxim Popov студентом

In [65]:
cur = conn.cursor()
cur.execute('''UPDATE users SET profile = (SELECT id FROM user_types WHERE name = "Student")
               WHERE first_name = "Maxim" AND last_name = "Popov"
''')
select_all_students_optimized(conn)

Maxim Popov
Ivan Ivanov
Petr Petrov
Petr Sidorov


In [66]:
conn.close()

### SQLAlchemy

<img src='pictures/sql_alchemy.png' />

In [67]:
import sqlalchemy
from sqlalchemy import create_engine

In [68]:
engine = create_engine('sqlite:///example_alchemy.db', echo=True)
# engine = create_engine('sqlite:///:memory:', echo=True)

#echo=True - для дополнительного логгирования

Опишем данные, которые хотим хранить

In [69]:
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Boolean
from sqlalchemy import ForeignKey
from sqlalchemy.orm import relationship

Base = declarative_base()

class UserType(Base):
    __tablename__ = 'user_types'
    
    id = Column(Integer, primary_key=True)
    name = Column(String)
    users = relationship("User")
    
    def __repr__(self):
        return "<UserType(name={})>".format(self.name)  

class User(Base):
    __tablename__ = 'users'
    
    id = Column(Integer, primary_key=True)
    first_name = Column(String)
    last_name = Column(String)
    active = Column(Boolean)
    profile_id = Column(Integer, ForeignKey('user_types.id'))
    profile = relationship("UserType", back_populates="users")
    
    def __repr__(self):
        return "<User(first_name={}, last_name={}, active={})>".format(self.first_name, self.last_name, self.active)
    

In [70]:
User.__table__

Table('users', MetaData(bind=None), Column('id', Integer(), table=<users>, primary_key=True, nullable=False), Column('first_name', String(), table=<users>), Column('last_name', String(), table=<users>), Column('active', Boolean(), table=<users>), Column('profile_id', Integer(), ForeignKey('user_types.id'), table=<users>), schema=None)

In [71]:
Base.metadata.create_all(engine)

2018-04-27 10:54:30,900 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2018-04-27 10:54:30,903 INFO sqlalchemy.engine.base.Engine ()
2018-04-27 10:54:30,908 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2018-04-27 10:54:30,911 INFO sqlalchemy.engine.base.Engine ()
2018-04-27 10:54:30,916 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("user_types")
2018-04-27 10:54:30,920 INFO sqlalchemy.engine.base.Engine ()
2018-04-27 10:54:30,925 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("users")
2018-04-27 10:54:30,927 INFO sqlalchemy.engine.base.Engine ()
2018-04-27 10:54:30,930 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE user_types (
	id INTEGER NOT NULL, 
	name VARCHAR, 
	PRIMARY KEY (id)
)


2018-04-27 10:54:30,932 INFO sqlalchemy.engine.base.Engine ()
2018-04-27 10:54:30,950 INFO sqlalchemy.engine.base.Engine COMMIT
2018-04-27 10:54:30,958 INFO sqlalchemy.engine.base.Engine 
CR

Вставим данные

In [72]:
from sqlalchemy.orm import sessionmaker

Session = sessionmaker(bind=engine)
session = Session()

teacher = UserType(name='teacher')
session.add(teacher)

student = UserType(name='student')
session.add(student)

session.commit()

2018-04-27 10:54:31,006 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-04-27 10:54:31,011 INFO sqlalchemy.engine.base.Engine INSERT INTO user_types (name) VALUES (?)
2018-04-27 10:54:31,014 INFO sqlalchemy.engine.base.Engine ('teacher',)
2018-04-27 10:54:31,020 INFO sqlalchemy.engine.base.Engine INSERT INTO user_types (name) VALUES (?)
2018-04-27 10:54:31,023 INFO sqlalchemy.engine.base.Engine ('student',)
2018-04-27 10:54:31,027 INFO sqlalchemy.engine.base.Engine COMMIT


Найдем тип учителя в базе

In [114]:
res = session.query(UserType).filter_by(name='teacher').first() 
res

2018-04-27 11:30:54,900 INFO sqlalchemy.engine.base.Engine SELECT user_types.id AS user_types_id, user_types.name AS user_types_name 
FROM user_types 
WHERE user_types.name = ?
 LIMIT ? OFFSET ?
2018-04-27 11:30:54,902 INFO sqlalchemy.engine.base.Engine ('teacher', 1, 0)


<UserType(name=teacher)>

In [115]:
teacher is res

True

one vs first

In [117]:
session.query(UserType).filter_by(name='teacher12').one() 

2018-04-27 11:31:24,361 INFO sqlalchemy.engine.base.Engine SELECT user_types.id AS user_types_id, user_types.name AS user_types_name 
FROM user_types 
WHERE user_types.name = ?
2018-04-27 11:31:24,363 INFO sqlalchemy.engine.base.Engine ('teacher12',)


NoResultFound: No row was found for one()

In [118]:
session.query(UserType).one() 

2018-04-27 11:31:36,679 INFO sqlalchemy.engine.base.Engine SELECT user_types.id AS user_types_id, user_types.name AS user_types_name 
FROM user_types
2018-04-27 11:31:36,681 INFO sqlalchemy.engine.base.Engine ()


MultipleResultsFound: Multiple rows were found for one()

Разные виды "select"

In [120]:
results = session.query(UserType).order_by(UserType.id).all()
print(type(results))
print(results)

2018-04-27 11:32:48,530 INFO sqlalchemy.engine.base.Engine SELECT user_types.id AS user_types_id, user_types.name AS user_types_name 
FROM user_types ORDER BY user_types.id
2018-04-27 11:32:48,532 INFO sqlalchemy.engine.base.Engine ()
<class 'list'>
[<UserType(name=teacher)>, <UserType(name=student)>]


In [121]:
for name, user_id in session.query(UserType.name, UserType.id).order_by(UserType.name):
    print(name, user_id)

2018-04-27 11:32:48,718 INFO sqlalchemy.engine.base.Engine SELECT user_types.name AS user_types_name, user_types.id AS user_types_id 
FROM user_types ORDER BY user_types.name
2018-04-27 11:32:48,719 INFO sqlalchemy.engine.base.Engine ()
student 2
teacher 1


Добавим нового учителя

In [122]:
#("Maxim", "Popov", 1, (SELECT id FROM user_types WHERE name = "Teacher")
user = User(first_name="Maxim", last_name="Popov", active=True)
teacher.users.append(user)
session.add(user)
session.commit()

2018-04-27 11:32:49,102 INFO sqlalchemy.engine.base.Engine INSERT INTO users (first_name, last_name, active, profile_id) VALUES (?, ?, ?, ?)
2018-04-27 11:32:49,104 INFO sqlalchemy.engine.base.Engine ('Maxim', 'Popov', 1, 1)
2018-04-27 11:32:49,107 INFO sqlalchemy.engine.base.Engine COMMIT


In [77]:
res = session.query(User).first() 
res

2018-04-27 10:56:12,429 INFO sqlalchemy.engine.base.Engine INSERT INTO users (first_name, last_name, active, profile_id) VALUES (?, ?, ?, ?)
2018-04-27 10:56:12,431 INFO sqlalchemy.engine.base.Engine ('Maxim', 'Popov', 1, 1)
2018-04-27 10:56:12,434 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.first_name AS users_first_name, users.last_name AS users_last_name, users.active AS users_active, users.profile_id AS users_profile_id 
FROM users
 LIMIT ? OFFSET ?
2018-04-27 10:56:12,435 INFO sqlalchemy.engine.base.Engine (1, 0)


<User(first_name=Maxim, last_name=Popov, active=True)>

In [84]:
res.profile

<UserType(name=teacher)>

Добавим студента

In [79]:
user = User(first_name="Petr", last_name="Sidorov", active=False, profile=student)
session.add(user)
session.commit()

2018-04-27 10:59:05,636 INFO sqlalchemy.engine.base.Engine SELECT user_types.id AS user_types_id, user_types.name AS user_types_name 
FROM user_types 
WHERE user_types.id = ?
2018-04-27 10:59:05,638 INFO sqlalchemy.engine.base.Engine (2,)
2018-04-27 10:59:05,641 INFO sqlalchemy.engine.base.Engine INSERT INTO users (first_name, last_name, active, profile_id) VALUES (?, ?, ?, ?)
2018-04-27 10:59:05,642 INFO sqlalchemy.engine.base.Engine ('Petr', 'Sidorov', 0, 2)
2018-04-27 10:59:05,644 INFO sqlalchemy.engine.base.Engine COMMIT


In [80]:
student.users

2018-04-27 11:00:00,441 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-04-27 11:00:00,443 INFO sqlalchemy.engine.base.Engine SELECT user_types.id AS user_types_id, user_types.name AS user_types_name 
FROM user_types 
WHERE user_types.id = ?
2018-04-27 11:00:00,445 INFO sqlalchemy.engine.base.Engine (2,)
2018-04-27 11:00:00,448 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.first_name AS users_first_name, users.last_name AS users_last_name, users.active AS users_active, users.profile_id AS users_profile_id 
FROM users 
WHERE ? = users.profile_id
2018-04-27 11:00:00,449 INFO sqlalchemy.engine.base.Engine (2,)


[<User(first_name=Petr, last_name=Sidorov, active=False)>]

Сделаем его учителем

In [81]:
user.profile = teacher
session.commit()

2018-04-27 11:02:57,968 INFO sqlalchemy.engine.base.Engine SELECT user_types.id AS user_types_id, user_types.name AS user_types_name 
FROM user_types 
WHERE user_types.id = ?
2018-04-27 11:02:57,970 INFO sqlalchemy.engine.base.Engine (1,)
2018-04-27 11:02:57,973 INFO sqlalchemy.engine.base.Engine UPDATE users SET profile_id=? WHERE users.id = ?
2018-04-27 11:02:57,975 INFO sqlalchemy.engine.base.Engine (1, 2)
2018-04-27 11:02:57,977 INFO sqlalchemy.engine.base.Engine COMMIT


In [82]:
student.users

2018-04-27 11:03:07,398 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-04-27 11:03:07,400 INFO sqlalchemy.engine.base.Engine SELECT user_types.id AS user_types_id, user_types.name AS user_types_name 
FROM user_types 
WHERE user_types.id = ?
2018-04-27 11:03:07,402 INFO sqlalchemy.engine.base.Engine (2,)
2018-04-27 11:03:07,404 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.first_name AS users_first_name, users.last_name AS users_last_name, users.active AS users_active, users.profile_id AS users_profile_id 
FROM users 
WHERE ? = users.profile_id
2018-04-27 11:03:07,405 INFO sqlalchemy.engine.base.Engine (2,)


[]

In [83]:
teacher.users

2018-04-27 11:03:13,171 INFO sqlalchemy.engine.base.Engine SELECT user_types.id AS user_types_id, user_types.name AS user_types_name 
FROM user_types 
WHERE user_types.id = ?
2018-04-27 11:03:13,173 INFO sqlalchemy.engine.base.Engine (1,)
2018-04-27 11:03:13,175 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.first_name AS users_first_name, users.last_name AS users_last_name, users.active AS users_active, users.profile_id AS users_profile_id 
FROM users 
WHERE ? = users.profile_id
2018-04-27 11:03:13,177 INFO sqlalchemy.engine.base.Engine (1,)


[<User(first_name=Maxim, last_name=Popov, active=True)>,
 <User(first_name=Petr, last_name=Sidorov, active=False)>]

Какая еще бывает фильтрация?

In [119]:
# equals
session.query(UserType).filter(UserType.name == 'teacher')
# not equals
session.query(UserType).filter(UserType.name != 'teacher')
# LIKE
session.query(UserType).filter(UserType.name.like('%tea%'))
# ILIKE (case-insensitive LIKE):
session.query(UserType).filter(UserType.name.ilike('%TeA%'))
# IS NULL
session.query(UserType).filter(UserType.name.is_(None))
# IS NOT NULL
session.query(UserType).filter(UserType.name.isnot(None))
              
from sqlalchemy import and_
#AND
session.query(UserType).filter(and_(UserType.name.isnot(None), UserType.name != 'abc'))
session.query(UserType).filter(UserType.name.isnot(None), UserType.name != 'abc')
session.query(UserType).filter(UserType.name.isnot(None)).filter(UserType.name != 'abc')

#OR
from sqlalchemy import or_
session.query(UserType).filter(or_(UserType.name.isnot(None), UserType.name != 'abc'))

In [127]:
#IN
session.query(User).filter(User.profile_id.in_(
    session.query(UserType.id)
)).all()

2018-04-27 11:38:10,395 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-04-27 11:38:10,398 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.first_name AS users_first_name, users.last_name AS users_last_name, users.active AS users_active, users.profile_id AS users_profile_id 
FROM users 
WHERE users.profile_id IN (SELECT user_types.id AS user_types_id 
FROM user_types)
2018-04-27 11:38:10,399 INFO sqlalchemy.engine.base.Engine ()


[<User(first_name=Maxim, last_name=Popov, active=True)>,
 <User(first_name=Petr, last_name=Sidorov, active=False)>,
 <User(first_name=Maxim, last_name=Popov, active=True)>]